# Conversation with AI - Chatbot

In [1]:
# Import libraries
import os
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr

import datetime
print(f"This notebook was last run at: {datetime.datetime.now() : %d/%m/%y %H:%M:%S.%f }")

This notebook was last run at:  25/09/25 23:24:05.212799 
